### 1.2 Trivial compression
---

Saving space (virtual or real) is often important. It is more efficent to use less space, and it can save money. If you are renting an apartment that is bigger than you need for your things and family, you could "downsize" to a smaller place that is less expensive. If you are paying by the byte to store your data on a server, you may want to compress it so that it storage costs you less. *Compression* is the act of taking data and encoding it (changing its form) in such a way that it takes up less space. *Decompression* is reversing the process, returning the data to its original form. 
    If it is more storage-efficient to compress data, then why is all data not compressed? There is a tradeoff between time and space. It takes time to compress a piece of data and to decompress it back into its original form. Therefore, data compression only makes sense in situations where small size is prioritized over fast execution. Think of large files being transmittied over the internet. Compressing them makes sense because it will take longer to transfer the files for their storage on the original server only needs to be accounted for once.
   
   The easiest data compression wins come about when you realize the data storate types use more bits than are strictly required for their contents. For instance, thinking low-level, if an unsigned integer that will never exceed 65,535 is being stored as a 64-bit unsigned integer in memory, it is being stored inefficiently. It could instead be stored as a 16-bit unsigned integer. This would reduce the space comsumption for the actual number by 75% (16 bits instead of 64 bits). If millions of such numbers are being stored inefficiently, it can add up to megabytes of wasted space.
    
   In Python, sometimes for the sake of simplicity, the developer is shielded from thinking in bits. There is no 64-bit unsigned integer type, and there is no 16-bit unsigned integer type. There is just a single int type that can store numbers of arbitrary precision. The function sys.getsizeof() can help you find out how many bytes of memory your Python objects are consuming. But due to the inherent overhead of the Python system, there is no way to create an int that takes up less than 28 bytes (224 bits) in Python 3.7. A single int can be extended one bit at a time (as we will do in this example), but it consumes a minimum of 28 bytes. 
    
   **Note**: If you are a little rust regarding binary, recall that a bit is a single value that is either a 1 or a 0. A sequence of 1s and 0s is read in base 2 to represent a number. 
   
If the number of possible different values that a type is meant to represent is less than the number of values that the bits being used to store it can represent, it can likely be more efficiently stored.

By converting letters of the four values of the nucleotides that form a DNA gene; each can represent G, A, C, T. Yet if the gene is stored as a string it can be represented as Unicode. Each Unicode character will generally require 8 bits of storage. 

A *CompressedGene* is provided a str of characters representing the nucleotides in a gene and it internall stores the squence of nucleotides as a bit string. The __init__() method's main responsibilitiy is to initilize the bit-string construct with the appropriate data. __init__() calls $_compress()$ to do the dirty work of actually converting the provided str of nucleotides into a bit string.

Note that $_compress()$ starts with an underscore. Python has no concept of truly prvate methods or variables. (All variables and methods can be accessed through reflection; there's no strict enforcement of privacy). A leading underscore is used as a convention to indicate that the implementation of a method should not be relied on by actors outside of the class. (It is subject to change and should be treated as private)

**TIP** If you start a method or instance variable name in a class with two leading underscores, Python will "name mangle" it, chaning its implementation name with a salt and not making it easily discoverable by other classes. We use one underscore in this book to indicate a "private" variable or method, by you may wish to use two if you really want to emphazise that something is private.

The $_compress()$ method looks at each character in the str of nucleotides sequentially. When it sees as A, it adds 00 to the bit string. When it sees a C, it adds 01, and so on. Remember that two bits are needed fro each nucleotide. As a result, before we add each new nucleotide, we shift the bit string two bits to the left (self.bit_string <<= 2).

Every nucleotide is added using an "or" operation (|). After the left shift, two 0s are added to the right  side of the bit string. In bitwise operations, "ORing" (for example, self.bit_string |= 0b10) 0s with any other value results in the other value replacing the 0s. In other words, we continually add two new bits to the right side of th bit string. The two bits that are added are determined by the type of the nucleotide. Finally, we will implement decompression and the special __str__() method that uses it.

In [8]:
from sys import getsizeof
import time

class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)
        
    def _compress(self, gene: str) -> None:
        self.bit_string: int = 1    # start with sentinel
        for nucleotide in gene.upper():
            self.bit_string <<= 2   # shift left two bits
            if nucleotide == "A":   # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C":
                self.bit_string |= 0b01
            elif nucleotide == "G":
                self.bit_string |= 0b10
            elif nucleotide == "T":
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid Nucleotide:{}".format(nucleotide))
                
    def _decompress(self) -> str:
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2):    # - 1 to exclude sentenials
            bits: int = self.bit_string >> i & 0b11    # get just 2 bits 
            if bits == 0b00: 
                gene += "A"
            elif bits == 0b01: 
                gene += "C"
            elif bits == 0b10:
                gene += "G"
            elif bits == 0b11:
                gene += "T"
            else:
                return ValueError("Invalid bits:{}".format(bits))
        return gene[::-1]   # returns the string in reverse by slicing backwards
    
    def __str__(self) -> str:
        return self._decompress()

original: str = "TAGGATTAACCGTTATATATATATATATGCTAGGATTATATTAATTAGGGAGAGA" * 100

start = time.time()
print("original is {} bytes".format(getsizeof(original)))
compressed: CompressedGene = CompressedGene(original)
print("compress is {} bytes". format(getsizeof(compressed.bit_string)))
# print(compressed)
print("original and decompressed are the same: {}".format(original == compressed._decompress()))
end = time.time()

print("running time", end-start)

AttributeError: 'builtin_function_or_method' object has no attribute 'time'

**Note**: In the CompressedGene class, we used if statements extensively to decide between a series of cases in bpth the compression and the decompression methods. Because Python has no switch statement, this is somewhat typeical. What you will also see in python sometimes is a high reliance on dictionaries in place of extensive if statements to deal with a set of cases. Imagine, for instance, a dictionary in which we could look up each nucleotidesrespective bits. The running time of this of a Dictionary was used is O(1)